In [1]:
import torch
from torch import nn
from collections import OrderedDict

def conv_batch(in_num, out_num, kernel_size=3, padding=1, stride=1):
    return nn.Sequential(
        nn.Conv2d(in_num, out_num, kernel_size=kernel_size, stride=stride, padding=padding, bias=False),
        nn.BatchNorm2d(out_num),
        nn.LeakyReLU())


# Residual block
class DarkResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(DarkResidualBlock, self).__init__()

        reduced_channels = int(in_channels/2)
        self.layer1 = conv_batch(in_channels, reduced_channels, kernel_size=1, padding=0)
        self.layer2 = conv_batch(reduced_channels, in_channels)

    def forward(self, x):
        residual = x

        out = self.layer1(x)
        out = self.layer2(out)
        out += residual
        return out


class Darknet53(nn.Module):
    def __init__(self, block, num_classes):
        super(Darknet53, self).__init__()

        self.num_classes = num_classes

        self.conv1 = conv_batch(3, 32)
        self.conv2 = conv_batch(32, 64, stride=2)
        self.residual_block1 = self.make_layer(block, in_channels=64, num_blocks=1)
        self.conv3 = conv_batch(64, 128, stride=2)
        self.residual_block2 = self.make_layer(block, in_channels=128, num_blocks=2)
        self.conv4 = conv_batch(128, 256, stride=2)
        self.residual_block3 = self.make_layer(block, in_channels=256, num_blocks=8)
        self.conv5 = conv_batch(256, 512, stride=2)
        self.residual_block4 = self.make_layer(block, in_channels=512, num_blocks=8)
        self.conv6 = conv_batch(512, 1024, stride=2)
        self.residual_block5 = self.make_layer(block, in_channels=1024, num_blocks=4)
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1024, self.num_classes)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.residual_block1(out)
        out = self.conv3(out)
        out = self.residual_block2(out)
        out = self.conv4(out)
        out = self.residual_block3(out)
        out = self.conv5(out)
        out = self.residual_block4(out)
        out = self.conv6(out)
        out = self.residual_block5(out)
        out = self.global_avg_pool(out)
        out = out.view(-1, 1024)
        out = self.fc(out)

        return out

    def represent_model(self) :
        self.features = nn.Sequential(
            self.conv1,
            self.conv2,
            self.residual_block1[0].layer1,
            self.residual_block1[0].layer2,
            self.conv3,
            self.residual_block2[0].layer1,
            self.residual_block2[0].layer2,
            self.residual_block2[1].layer1,
            self.residual_block2[1].layer2,
            self.conv4,
            self.residual_block3[0].layer1,
            self.residual_block3[0].layer2,
            self.residual_block3[1].layer1,
            self.residual_block3[1].layer2,
            self.residual_block3[2].layer1,
            self.residual_block3[2].layer2,
            self.residual_block3[3].layer1,
            self.residual_block3[3].layer2,
            self.residual_block3[4].layer1,
            self.residual_block3[4].layer2,
            self.residual_block3[5].layer1,
            self.residual_block3[5].layer2,
            self.residual_block3[6].layer1,
            self.residual_block3[6].layer2,
            self.residual_block3[7].layer1,
            self.residual_block3[7].layer2,
            self.conv5,
            self.residual_block4[0].layer1,
            self.residual_block4[0].layer2,
            self.residual_block4[1].layer1,
            self.residual_block4[1].layer2,
            self.residual_block4[2].layer1,
            self.residual_block4[2].layer2,
            self.residual_block4[3].layer1,
            self.residual_block4[3].layer2,
            self.residual_block4[4].layer1,
            self.residual_block4[4].layer2,
            self.residual_block4[5].layer1,
            self.residual_block4[5].layer2,
            self.residual_block4[6].layer1,
            self.residual_block4[6].layer2,
            self.residual_block4[7].layer1,
            self.residual_block4[7].layer2,
            self.conv6,
            self.residual_block5[0].layer1,
            self.residual_block5[0].layer2,
            self.residual_block5[1].layer1,
            self.residual_block5[1].layer2,
            self.residual_block5[2].layer1,
            self.residual_block5[2].layer2,
            self.residual_block5[3].layer1,
            self.residual_block5[3].layer2)

        self.classifier = nn.Sequential(nn.Sequential(self.global_avg_pool, self.fc))

    def make_layer(self, block, in_channels, num_blocks):
        layers = []
        for i in range(0, num_blocks):
            layers.append(block(in_channels))
        return nn.Sequential(*layers)

    def darknet53(num_classes):
        return Darknet53(DarkResidualBlock, num_classes)

In [2]:
model = Darknet53.darknet53(1000)

In [3]:
model = model.eval()

In [4]:
import numpy as np

In [5]:
checkpoint = torch.load("model_best.pth.tar", map_location=torch.device('cpu'))

In [6]:
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [7]:
model.represent_model()

In [8]:
input_tensor = torch.FloatTensor(np.array([1.0] * 224 * 224 * 3))
input_tensor = input_tensor.reshape([1, 3, 224, 224])

In [9]:
idx = 0
def get_output(layer, eval = True) :
    if eval:
        layer = layer.eval()
    global input_tensor, idx
    input_tensor = layer(input_tensor)
    idx += 1
    print("Output of layer ", idx, " : ", input_tensor.sum())

In [10]:
def reset():
    global input_tensor, idx
    input_tensor = torch.FloatTensor(np.array([1.0] * 224 * 224 * 3))
    input_tensor = input_tensor.reshape([1, 3, 224, 224])
    idx = 0

In [11]:
model = model.eval()
reset()
for layer in model.conv1:
    get_output(layer)

Output of layer  1  :  tensor(-66494.6406, grad_fn=<SumBackward0>)
Output of layer  2  :  tensor(91958.9922, grad_fn=<SumBackward0>)
Output of layer  3  :  tensor(213123.9062, grad_fn=<SumBackward0>)


In [12]:
for layer in model.conv2:
    get_output(layer)

Output of layer  4  :  tensor(-60583.6055, grad_fn=<SumBackward0>)
Output of layer  5  :  tensor(28021.8281, grad_fn=<SumBackward0>)
Output of layer  6  :  tensor(88063.1875, grad_fn=<SumBackward0>)


In [13]:
for layer in model.residual_block1:
    get_output(layer)

Output of layer  7  :  tensor(178315.9844, grad_fn=<SumBackward0>)


In [14]:
for layer in model.conv3:
    get_output(layer)

Output of layer  8  :  tensor(-27250.8750, grad_fn=<SumBackward0>)
Output of layer  9  :  tensor(6702.3643, grad_fn=<SumBackward0>)
Output of layer  10  :  tensor(14910.2334, grad_fn=<SumBackward0>)


In [15]:
for layer in model.residual_block2:
    get_output(layer)

Output of layer  11  :  tensor(29782.5117, grad_fn=<SumBackward0>)
Output of layer  12  :  tensor(36002.1445, grad_fn=<SumBackward0>)


In [16]:
for layer in model.conv4:
    get_output(layer)

Output of layer  13  :  tensor(-18968.7227, grad_fn=<SumBackward0>)
Output of layer  14  :  tensor(5787.4888, grad_fn=<SumBackward0>)
Output of layer  15  :  tensor(14683.9355, grad_fn=<SumBackward0>)


In [17]:
for layer in model.residual_block3:
    get_output(layer)

Output of layer  16  :  tensor(16319.0957, grad_fn=<SumBackward0>)
Output of layer  17  :  tensor(18173.0469, grad_fn=<SumBackward0>)
Output of layer  18  :  tensor(20638.9766, grad_fn=<SumBackward0>)
Output of layer  19  :  tensor(22421.3496, grad_fn=<SumBackward0>)
Output of layer  20  :  tensor(24824.5801, grad_fn=<SumBackward0>)
Output of layer  21  :  tensor(28561.0898, grad_fn=<SumBackward0>)
Output of layer  22  :  tensor(30829.2363, grad_fn=<SumBackward0>)
Output of layer  23  :  tensor(36881.2695, grad_fn=<SumBackward0>)


In [18]:
for layer in model.conv5:
    get_output(layer)

Output of layer  24  :  tensor(-21828.9863, grad_fn=<SumBackward0>)
Output of layer  25  :  tensor(-6754.0391, grad_fn=<SumBackward0>)
Output of layer  26  :  tensor(3080.4380, grad_fn=<SumBackward0>)


In [19]:
for layer in model.residual_block4:
    get_output(layer)

Output of layer  27  :  tensor(3371.2676, grad_fn=<SumBackward0>)
Output of layer  28  :  tensor(4013.7974, grad_fn=<SumBackward0>)
Output of layer  29  :  tensor(4991.7456, grad_fn=<SumBackward0>)
Output of layer  30  :  tensor(6286.2559, grad_fn=<SumBackward0>)
Output of layer  31  :  tensor(7087.5371, grad_fn=<SumBackward0>)
Output of layer  32  :  tensor(9094.2822, grad_fn=<SumBackward0>)
Output of layer  33  :  tensor(11172.4922, grad_fn=<SumBackward0>)
Output of layer  34  :  tensor(15173.5615, grad_fn=<SumBackward0>)


In [20]:
for layer in model.conv6:
    get_output(layer)

Output of layer  35  :  tensor(-5633.3867, grad_fn=<SumBackward0>)
Output of layer  36  :  tensor(-4110.5840, grad_fn=<SumBackward0>)
Output of layer  37  :  tensor(1388.2852, grad_fn=<SumBackward0>)


In [21]:
for layer in model.residual_block5:
    get_output(layer)

Output of layer  38  :  tensor(1942.5583, grad_fn=<SumBackward0>)
Output of layer  39  :  tensor(2399.0015, grad_fn=<SumBackward0>)
Output of layer  40  :  tensor(3401.9531, grad_fn=<SumBackward0>)
Output of layer  41  :  tensor(7979.6392, grad_fn=<SumBackward0>)


In [22]:
input_tensor = model.global_avg_pool(input_tensor)

In [25]:
print(input_tensor.sum())

tensor(162.8498, grad_fn=<SumBackward0>)


In [32]:
input_tensor[-1, 101:501].sum()

tensor(62.7226, grad_fn=<SumBackward0>)

In [66]:
input_tensor.view(-1, 1024).shape

torch.Size([1, 1024])

In [33]:
input_tensor = input_tensor.view(-1, 1024)
model.fc(input_tensor).sum()

tensor(6.4653, grad_fn=<SumBackward0>)

In [34]:
model.fc(input_tensor).max()


tensor(3.6737, grad_fn=<MaxBackward1>)

In [35]:
model.fc(input_tensor).argmax()

tensor(623)

In [34]:
reset()
model.eval()
model(input_tensor).sum()

tensor(6.4653, grad_fn=<SumBackward0>)

In [69]:
from torchvision import transforms
from PIL import Image 
model.eval()
classes = os.listdir("../imagenette-test")
for folder in classes :
    print(folder)
    images = os.listdir("../imagenette-test/" + folder)
    for image in images:
        train_transforms = transforms.Compose([transforms.Resize(224), transforms.ToTensor()])
        img = Image.open("../imagenette-test/" + folder + "/" + image)
        img = train_transforms(img)
        temp = img
        img = img.unsqueeze(0)

        index = torch.argmax(model(img))
        print(index, " ---->", model(img)[0][index], " ---> ", image, "------>" ,img.sum())
        mat.append(img.numpy().ravel())

n03394916
tensor(601)  ----> tensor(8.7842, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00003759.jpg ------> tensor(68339.8203)
tensor(566)  ----> tensor(13.2638, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00005548.jpg ------> tensor(25738.9199)
tensor(566)  ----> tensor(15.0629, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00007985.jpg ------> tensor(46697.1094)
tensor(513)  ----> tensor(24.3026, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00005554.jpg ------> tensor(19992.7109)
tensor(566)  ----> tensor(17.3594, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00007155.jpg ------> tensor(25416.3613)
tensor(566)  ----> tensor(12.3731, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00000957.jpg ------> tensor(25532.3027)
tensor(513)  ----> tensor(10.9770, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00003587.jpg ------> tensor(68653.9297)
tensor(566)  ----> tensor(11.1971, grad_fn=<SelectBackward>)  --->  ILSVRC2012_val_00007536.jpg ------> tensor(34702.4531)
n034170

In [65]:
import torch
from torch import nn
import os
import numpy as np
import argparse
from xml.etree import ElementTree

def make_directory(base_path : str) -> int :
    """
        Checks if a directory exists and if doesn't creates the directory.

        Args:
        base_path : Directory path which will be created if it doesn't exist.

        Returns 0 if directory exists else 1
    """
    if os.path.exists(base_path) :
        return 0

    # Create the directory since the path doesn't exist.
    os.mkdir(base_path)
    if os.path.exists(base_path) :
        return 0

    # Path doesn't exist as well as directory couldn't be created.
    print("Error : Cannot create desired path : ", base_path)
    return 1

def generate_csv(csv_name : str, weight_matrix : torch.tensor, base_path : str) -> str :
    """
        Generates csv for weights or bias matrix.

        Args:
        csv_name : A string name for csv file which will store the weights.
        weight_matrix : A torch tensor holding weights that will be stored in the matrix.
        base_path : Base path where csv will be stored.
    """
    # Check if base path exists else create directory.
    make_directory(base_path)
    file_path = os.path.join(base_path, csv_name)
    matrix = weight_matrix.numpy().ravel()
    np.savetxt(file_path, matrix, fmt='%1.128f')
    return file_path

def extract_weights(layer, layer_index, base_path) -> {} :
    """
        Extracts weights, biases and other parameters required to reproduce
        the same output.

        Args:
        layer : An torch.nn object (layer).
        layer_index : A string determining name of csv file that will be appended to
                      name of layer.
                      Eg. if layer = nn.Conv2d and layer_index = 0
                          csv_filename = Conv_layer_index.csv
        base_path : A string depicting base path for storing weight / bias csv.

        Returns dictionary of parameter description and parameters.

        Exceptions:
        Currently this has only been tested for convolutional and batch-norm layer.
    """
    parameter_dictionary = {}
    if isinstance(layer, nn.Conv2d):
        # The layer corresponds to Convolutional layer.
        # For convolution layer we require weights and biases to reproduce the
        # same result.
        parameter_dictionary["name"] = "Convolution2D"
        parameter_dictionary["input-channels"] = layer.in_channels
        parameter_dictionary["output-channels"] = layer.out_channels
        # Assume weight matrix is never empty for nn.Conv2d()
        parameter_dictionary["has_weights"] = 1
        parameter_dictionary["weight_offset"] = 0
        csv_name = "conv_weight_" + layer_index + ".csv"
        parameter_dictionary["weight_csv"] = generate_csv(csv_name, \
            layer.weight.detach(), base_path)
        if layer.bias != None:
            parameter_dictionary["has_bias"] = 1
            parameter_dictionary["bias_offset"] = 0
            bias_csv_name = "conv_bias_" + layer_index + ".csv"
            parameter_dictionary["bias_csv"] = generate_csv(bias_csv_name, \
                layer.bias.detach(), base_path)
        else:
            parameter_dictionary["has_bias"] = 0
            parameter_dictionary["bias_offset"] = layer.out_channels
            parameter_dictionary["bias_csv"] = "None"
        parameter_dictionary["has_running_mean"] = 0
        parameter_dictionary["running_mean_csv"] = "None"
        parameter_dictionary["has_running_var"] = 0
        parameter_dictionary["running_var_csv"] = "None"
    elif isinstance(layer, nn.BatchNorm2d) :
        # The layer corresponds to Batch Normalization layer.
        # For batchnorm layer we require weights, biases and running mean and running variance
        # to reproduce the same result.
        parameter_dictionary["name"] = "BatchNorm2D"
        parameter_dictionary["input-channels"] = layer.num_features
        parameter_dictionary["output-channels"] = layer.num_features
        # Assume weight matrix is never empty for nn.BatchNorm2d()
        parameter_dictionary["has_weights"] = 1
        parameter_dictionary["weight_offset"] = 0
        csv_name = "batchnorm_weight_" + layer_index + ".csv"
        parameter_dictionary["weight_csv"] = generate_csv(csv_name, \
            layer.weight.detach(), base_path)
        if layer.bias != None:
            parameter_dictionary["has_bias"] = 1
            parameter_dictionary["bias_offset"] = 0
            bias_csv_name = "batchnorm_bias_" + layer_index + ".csv"
            parameter_dictionary["bias_csv"] = generate_csv(bias_csv_name, \
                layer.bias.detach(), base_path)
        else:
            parameter_dictionary["has_bias"] = 0
            parameter_dictionary["bias_offset"] = layer.out_channels
            parameter_dictionary["bias_csv"] = "None"
        # Assume BatchNorm layer always running variance and running mean.
        running_mean_csv = "batchnorm_running_mean_" + layer_index + ".csv"
        parameter_dictionary["has_running_mean"] = 1
        parameter_dictionary["running_mean_csv"] = generate_csv(running_mean_csv, \
            layer.running_mean.detach(), base_path)
        parameter_dictionary["has_running_var"] = 1
        running_var_csv = "batchnorm_running_var_" + layer_index + ".csv" 
        parameter_dictionary["running_var_csv"] = generate_csv(running_var_csv, \
            layer.running_var.detach(), base_path)
    elif (isinstance(layer, nn.Linear)) :
        # The layer corresponds to Convolutional layer.
        # For convolution layer we require weights and biases to reproduce the
        # same result.
        parameter_dictionary["name"] = "Linear"
        parameter_dictionary["input-channels"] = layer.in_features
        parameter_dictionary["output-channels"] = layer.out_features
        # Assume weight matrix is never empty for nn.Linear()
        parameter_dictionary["has_weights"] = 1
        parameter_dictionary["weight_offset"] = 0
        csv_name = "linear_weight_" + layer_index + ".csv"
        parameter_dictionary["weight_csv"] = generate_csv(csv_name, \
            layer.weight.detach(), base_path)
        if layer.bias != None:
            parameter_dictionary["has_bias"] = 1
            parameter_dictionary["bias_offset"] = 0
            bias_csv_name = "linear_bias_" + layer_index + ".csv"
            parameter_dictionary["bias_csv"] = generate_csv(bias_csv_name, \
                layer.bias.detach(), base_path)
        else:
            parameter_dictionary["has_bias"] = 0
            parameter_dictionary["bias_offset"] = layer.out_features
            parameter_dictionary["bias_csv"] = "None"
        parameter_dictionary["has_running_mean"] = 0
        parameter_dictionary["running_mean_csv"] = "None"
        parameter_dictionary["has_running_var"] = 0
        parameter_dictionary["running_var_csv"] = "None"
    else :
        # The layer corresponds to un-supported layer or layer doesn't have trainable
        # parameter. Example of such layers are nn.MaxPooling2d() and nn.SoftMax.
        parameter_dictionary["name"] = "unknown_layer"
        parameter_dictionary["input-channels"] = 0
        parameter_dictionary["output-channels"] = 0
        parameter_dictionary["has_weights"] = 0
        parameter_dictionary["weight_offset"] = 0
        parameter_dictionary["weight_csv"] = "None"
        parameter_dictionary["has_bias"] = 0
        parameter_dictionary["bias_offset"] = 0
        parameter_dictionary["bias_csv"] = "None"
        parameter_dictionary["has_running_mean"] = 0
        parameter_dictionary["running_mean_csv"] = "None"
        parameter_dictionary["has_running_var"] = 0
        parameter_dictionary["running_var_csv"] = "None"
    return parameter_dictionary

def create_xml_tree(parameter_dictionary : dict, root_tag = "layer") -> ElementTree.ElementTree() :
    """
        Creates an XML tree from a dictionary wrapped around root tag.

        Args:
        parameter_dictionary : Dictionary which will be converted to xml tree.
        root_tag : Tag around which elements of dictionary will be wrapped.
                    Defaults to "layer".
    
        Returns : ElementTree.ElementTree() object.
    """
    layer = ElementTree.Element(root_tag)
    for parameter_desc in parameter_dictionary :
        parameter_description = ElementTree.Element(parameter_desc)
        parameter_description.text = str(parameter_dictionary[parameter_desc])
        layer.append(parameter_description)
    return layer

def create_xml_file(parameter_dictionary : dict,
                    xml_path : str,
                    root_tag : str,
                    element_tag : str) -> int :
    """
        Appends layer description to xml file and if xml doesn't exist or is empty, 
        creates an xml file with required headers.

        Args:
        parameter_dictionary : Dictionary containing layer description.
        xml_path : Path where xml file will be stored / created.
        root_tag : Tag around which xml file will be wrapped.
        element_tag : Tag around which each element in dictionary will be wrapped.
    """
   
    if not os.path.exists(xml_path) :
        # Create base xml file.
        f = open(xml_path, "w")
        data = "<" + root_tag + ">" + "</" + root_tag + ">"
        f.write(data)
        f.close()
    layer_description = create_xml_tree(parameter_dictionary, element_tag)
    xml_file = ElementTree.parse(xml_path)
    root = xml_file.getroot()
    layer = root.makeelement(element_tag, parameter_dictionary)
    root.append(layer_description)
    xml_file.write(xml_path, encoding = "unicode")
    return 0

def iterate_over_layers(modules, xml_path, base_path, layer_index, debug : bool) -> int :
    """
        Parses model and generates csv and xml file which will be iterated by C++ translator.
    
        Args:
        modules : PyTorch model for which parameter csv and xml will be created.
        xml_path : Directory where xml with model config will be saved.
        base_path : Directory where csv will be stored.

        Returns 0 if weights are created else return 1.
    """
    for block in modules :
        for layer in block :
            layer_index += 1
            parameter_dict = extract_weights(layer, str(layer_index), base_path)
            create_xml_file(parameter_dict, xml_path, "model", "layer")
            if not os.path.exists(parameter_dict["weight_csv"]) and parameter_dict["has_weights"] == 1:
                print("Creating weights failed!")
                return 1, layer_index
            if debug :
                print("Weights created succesfully for ", parameter_dict["name"], " layer index :", layer_index)
    return 0, layer_index

def parse_model(model, xml_path, base_path, debug : bool) -> int :
    """
        Parses model and generates csv and xml file which will be iterated by C++ translator.
    
        Args:
        model : PyTorch model for which parameter csv and xml will be created.
        xml_path : Directory where xml with model config will be saved.
        base_path : Directory where csv will be stored.

        Returns 0 if weights are created else return 1.
    """
    layer_index = 0
    error, layer_index = iterate_over_layers(model.features, xml_path, base_path, layer_index, debug)
    if error :
        print("An error occured!")
        return 1
    print(layer_index)
    error, layer_index = iterate_over_layers(model.classifier, xml_path, base_path, layer_index, debug)
    if error :
        print("An error occured!")
        return 1
    print(layer_index)
    if debug :
        print("Model weights saved! Happy mlpack-translation.")
    return 0

In [66]:
model = model.eval()
model.represent_model()

In [67]:
parse_model(model, "./cfg/" + "darknet53" + ".xml", "./models/" + "darknet53" + "/mlpack-weights/", True)

Weights created succesfully for  Convolution2D  layer index : 1
Weights created succesfully for  BatchNorm2D  layer index : 2
Weights created succesfully for  unknown_layer  layer index : 3
Weights created succesfully for  Convolution2D  layer index : 4
Weights created succesfully for  BatchNorm2D  layer index : 5
Weights created succesfully for  unknown_layer  layer index : 6
Weights created succesfully for  Convolution2D  layer index : 7
Weights created succesfully for  BatchNorm2D  layer index : 8
Weights created succesfully for  unknown_layer  layer index : 9
Weights created succesfully for  Convolution2D  layer index : 10
Weights created succesfully for  BatchNorm2D  layer index : 11
Weights created succesfully for  unknown_layer  layer index : 12
Weights created succesfully for  Convolution2D  layer index : 13
Weights created succesfully for  BatchNorm2D  layer index : 14
Weights created succesfully for  unknown_layer  layer index : 15
Weights created succesfully for  Convolution

0

In [19]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [20]:
get_n_params(model)

41609928

In [33]:
def conv_batch(in_num, out_num, kernel_size=3, padding=1, stride=1):
    return nn.Sequential(
        nn.Conv2d(in_num, out_num, kernel_size=kernel_size, stride=stride, padding=padding, bias=True),
        nn.BatchNorm2d(out_num),
        nn.LeakyReLU())
model2 = Darknet53.darknet53(1000)

In [34]:
get_n_params(model2)

41627784

In [40]:
!python3 -m pip install torchsummary

In [42]:
from torchsummary import summary

In [44]:
summary(model2, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             896
       BatchNorm2d-2         [-1, 32, 224, 224]              64
         LeakyReLU-3         [-1, 32, 224, 224]               0
            Conv2d-4         [-1, 64, 112, 112]          18,496
       BatchNorm2d-5         [-1, 64, 112, 112]             128
         LeakyReLU-6         [-1, 64, 112, 112]               0
            Conv2d-7         [-1, 32, 112, 112]           2,080
       BatchNorm2d-8         [-1, 32, 112, 112]              64
         LeakyReLU-9         [-1, 32, 112, 112]               0
           Conv2d-10         [-1, 64, 112, 112]          18,496
      BatchNorm2d-11         [-1, 64, 112, 112]             128
        LeakyReLU-12         [-1, 64, 112, 112]               0
DarkResidualBlock-13         [-1, 64, 112, 112]               0
           Conv2d-14          [-1, 128,

In [53]:
nn.Linear(122, 12).out_features

12

In [72]:
 model(torch.FloatTensor(np.array([1] * 224 * 224 * 3).reshape(3, 224, 224)).unsqueeze(0)).argmax()

tensor(623)